In [1]:
import torch
from torch import nn

import torchvision
from torchvision.transforms import v2
from torchvision import datasets

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

print(device)

cuda


In [5]:
root = '/content/drive/MyDrive/Colab Notebooks/DL_impl/ResNet/datasets'

train_ds = datasets.CIFAR10(root, train=True, transform=v2.ToTensor(), download=True)
test_ds = datasets.CIFAR10(root, train=False, transform=v2.ToTensor(), download=True)

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/v2/_deprecated.py:43: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.
  warnings.warn(


Files already downloaded and verified
Files already downloaded and verified


In [6]:
# dataset 크기 확인
print(f'train dataset size: {len(train_ds)}')
print(f'test dataset size: {len(test_ds)}')

train dataset size: 50000
test dataset size: 10000


In [7]:
class ResBlock(nn.Module):
  def __init__(self, in_channel, out_channel, downsample):
    super().__init__()

    self.stride = (2, 1) if downsample else (1, 1)
    self.conv1 = nn.Conv2d(in_channel, out_channel, kernel_size=3, stride=self.stride[0], padding=1)
    self.bn1 = nn.BatchNorm2d(out_channel)
    self.relu1 = nn.ReLU()
    self.conv2 = nn.Conv2d(out_channel, out_channel, kernel_size=3, stride=self.stride[1], padding=1)
    self.bn2 = nn.BatchNorm2d(out_channel)
    self.relu2 = nn.ReLU()

    self.downsample = None if not downsample else nn.Sequential(
        nn.Conv2d(in_channel, out_channel, kernel_size=1, stride=2),
        nn.BatchNorm2d(out_channel)
    )

  def forward(self, x):
    identity = x if not self.downsample else self.downsample(x)
    res = self.conv1(x)
    res = self.bn1(res)
    res = self.relu1(res)
    res = self.conv2(res)
    res = self.bn2(res)
    out = res + identity
    out = self.relu2(out)

    return out

In [8]:
class ResNet34(nn.Module):
  def __init__(self, block):
    super().__init__()

    self.input_layer = nn.Sequential(
        nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
    )

    self.layer1 = self._make_layer(block, 64, 64, 3)   # 이것만 downsampling 없음. 특징: in_channel == out_channel
    self.layer2 = self._make_layer(block, 64, 128, 4)
    self.layer3 = self._make_layer(block, 128, 256, 6)
    self.layer4 = self._make_layer(block, 256, 512, 3)

    self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
    self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(512, 1000)
    )

  def _make_layer(self, block, in_channel, out_channel, n_blocks):
    layers = []

    # 각 layer의 첫 번째 block
    # 첫 번째 layer만 시작하는 블록의 filter가 stride=1
    if in_channel == out_channel:
      layers.append(block(in_channel, out_channel, downsample=False))
    # 나머지 layer는 시작하는 블록의 filter가 strdie=2
    else:
      layers.append(block(in_channel, out_channel, downsample=True))

    for _ in range(1, n_blocks):
      layers.append(block(out_channel, out_channel, downsample=False))

    return nn.Sequential(*layers)

  def forward(self, x):
    x = self.input_layer(x)
    x = self.layer1(x)
    x = self.layer2(x)
    x = self.layer3(x)
    x = self.layer4(x)
    x = self.avgpool(x)
    x = self.classifier(x)

    return x

In [9]:
model = ResNet34(ResBlock).to(device)

In [10]:
print(model)

ResNet34(
  (input_layer): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (layer1): Sequential(
    (0): ResBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU()
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu2): ReLU()
    )
    (1): ResBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU()
      (conv2): Conv2d(64, 64, kernel_s

In [13]:
!pip install torchinfo

In [14]:
import torchinfo

batch_size = 256
torchinfo.summary(model, input_size=(batch_size, 3, 224, 224))

Layer (type:depth-idx)                   Output Shape              Param #
ResNet34                                 [256, 1000]               --
├─Sequential: 1-1                        [256, 64, 56, 56]         --
│    └─Conv2d: 2-1                       [256, 64, 112, 112]       9,472
│    └─BatchNorm2d: 2-2                  [256, 64, 112, 112]       128
│    └─ReLU: 2-3                         [256, 64, 112, 112]       --
│    └─MaxPool2d: 2-4                    [256, 64, 56, 56]         --
├─Sequential: 1-2                        [256, 64, 56, 56]         --
│    └─ResBlock: 2-5                     [256, 64, 56, 56]         --
│    │    └─Conv2d: 3-1                  [256, 64, 56, 56]         36,928
│    │    └─BatchNorm2d: 3-2             [256, 64, 56, 56]         128
│    │    └─ReLU: 3-3                    [256, 64, 56, 56]         --
│    │    └─Conv2d: 3-4                  [256, 64, 56, 56]         36,928
│    │    └─BatchNorm2d: 3-5             [256, 64, 56, 56]         128
│